In [1]:
from pyomo.environ import (Constraint,
                           Var,
                           ConcreteModel,
                           Expression,
                           Param,
                           Objective,
                           SolverFactory,
                           TransformationFactory,
                           value)

from pyomo.network import Arc, SequentialDecomposition

from idaes.core.util.initialization import propagate_state

# Import plotting functions
import matplotlib.pyplot as plt

# Import numpy library 
import numpy as np

import pandas as pd

# Import the main FlowsheetBlock from IDAES. The flowsheet block will contain the unit model
from idaes.core import FlowsheetBlock

import idaes.logger as idaeslog
from pyomo.opt import TerminationCondition, SolverStatus

# Import the degrees_of_freedom function from the idaes.core.util.model_statistics package
# DOF = Number of Model Variables - Number of Model Constraints
from idaes.core.util.model_statistics import degrees_of_freedom

# Import the Generic Parameter Block
from idaes.generic_models.properties.core.generic.generic_property import (
        GenericParameterBlock)

# Import unit models from the model library
from idaes.generic_models.unit_models import Flash

In [2]:
from R32_bmimPF6_PR import configuration

In [3]:
data = pd.read_csv('R32IL.csv')

solver = SolverFactory('ipopt')
solver.options = {'tol': 1e-6}

In [4]:
m = ConcreteModel()

m.fs = FlowsheetBlock(default={"dynamic": False})

m.fs.properties = GenericParameterBlock(default=configuration)

m.fs.F101 = Flash(default={"property_package": m.fs.properties,
                           "has_heat_transfer": True,
                           "has_pressure_change": True})

m.fs.F101.inlet.flow_mol.fix(1)
m.fs.F101.inlet.temperature.fix(323.15)
m.fs.F101.inlet.pressure.fix(399800)
m.fs.F101.inlet.mole_frac_comp[0,'R32'].fix(0.5)

m.fs.properties.PR_kappa_A['bmimPF6', 'R32'].fix(-0.10943851272230938)
m.fs.properties.PR_kappa_A['R32', 'bmimPF6'].fix(-0.05450860738318672)
m.fs.properties.PR_kappa_B['bmimPF6', 'R32'].fix(0.0)
m.fs.properties.PR_kappa_B['R32', 'bmimPF6'].fix(0.0)

# m.fs.properties.PR_kappa_A['bmimPF6', 'R32'].fix(-0.07529269623484444)
# m.fs.properties.PR_kappa_A['R32', 'bmimPF6'].fix(-0.04776098298737627)
# m.fs.properties.PR_kappa_B['bmimPF6', 'R32'].fix(0.0)
# m.fs.properties.PR_kappa_B['R32', 'bmimPF6'].fix(0.0)

# Initialize the flash unit
m.fs.F101.initialize(outlvl=idaeslog.CRITICAL)

# Fix the state variables on the state block
# m.fs.F101.inlet.pressure.unfix()
m.fs.F101.inlet.mole_frac_comp[0,'bmimPF6'].unfix()

R32_x = np.zeros((len(data)))
P = np.zeros((len(data)))

m.fs.liq = Param(mutable=True,default=0.040)
m.fs.liquid = Constraint(expr=m.fs.F101.liq_outlet.mole_frac_comp[0, "bmimPF6"] == m.fs.liq)

m.fs.liq = data["x_bmimPF6"].iloc[6]
m.fs.F101.inlet.temperature.fix(float(data["temperature"].iloc[6]))
m.fs.F101.inlet.pressure.fix(float(data["pressure"].iloc[6]))
m.fs.F101.inlet.mole_frac_comp[0,'R32'].fix(float(data["x_R32"].iloc[6])+0.1)
m.fs.F101.inlet.mole_frac_comp[0,'bmimPF6'].fix(float(1-(data["x_R32"].iloc[6]+0.1)))
m.fs.F101.vap_outlet.temperature.fix(float(data["temperature"].iloc[6]))
    
print("x =", data["x_bmimPF6"].iloc[6])


m.fs.F101.initialize(outlvl=idaeslog.CRITICAL)

status = solver.solve(m, tee = True)



x = 0.185
Ipopt 3.12.13: tol=1e-06


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collection of Fortran codes
    for large-scale scientific computation.  All technical papers, sales and
    publicity material resulting from use of the HSL codes within IPOPT must
    contain the following acknowledgement:
        HSL, a collection of Fortran codes for large-scale scientific
        com

In [5]:
m.fs.F101.report()


Unit : fs.F101                                                             Time: 0.0
------------------------------------------------------------------------------------
    Unit Performance

    Variables: 

    Key             : Value   : Fixed : Bounds
          Heat Duty : -1684.0 : False : (None, None)
    Pressure Change :  78638. : False : (None, None)

------------------------------------------------------------------------------------
    Stream Table
                              Inlet    Vapor Outlet  Liquid Outlet
    flow_mol                   1.0000      0.54054       0.45946  
    mole_frac_comp bmimPF6   0.085000   1.0000e-08       0.18500  
    mole_frac_comp R32        0.91500       1.0000       0.81500  
    temperature                283.20       283.20        283.20  
    pressure               8.4950e+05   9.2814e+05    9.2814e+05  
